<a href="https://colab.research.google.com/github/ssosoo/2024_DS60/blob/main/%ED%8C%8C%EC%9D%B4%ED%86%A0%EC%B9%98/%ED%85%8D%EC%8A%A4%ED%8A%B8_%EB%8D%B0%EC%9D%B4%ED%84%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

#순전파와 역전파, 모델 구조와 초깃값
class CustomModel(nn.Module):
  def __init__(self):
    super().__init__()

    self.layer1 = nn.Sequential(
        nn.Linear(2, 2),
        nn.Sigmoid()
    )

    self.layer2 = nn.Sequential(
        nn.Linear(2, 1),
        nn.Sigmoid()
    )

    self.layer1[0].weight.data = torch.nn.Parameter(
        torch.Tensor([[0.4352, 0.3545], [0.1951, 0.4835]])
    )

    self.layer1[0].bias.data = torch.nn.Parameter(
        torch.Tensor([-0.1419, 0.0439])
    )

    self.layer2[0].weight.data = torch.nn.Parameter(
        torch.Tensor([[-0.1725, 0.1129]])
    )

    self.layer2[0].bias.data = torch.nn.Parameter(
        torch.Tensor([-0.3043])
    )
    #bias는 선형 구조에 영향...받는다.

device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.BCELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=1)

**텍스트 데이터**

텍스트 데이터 증강은 자연어 처리 모델을 구성할 때 데이터세트의 크기를 조정한다.
삽입, 삭제, 교체, 대체, 생성, 반의어, 맞춤법 교정, 역번역 등이 있다.

In [2]:
#자연어 처리 데이터 증강 라이브러리
!pip install numpy requests nlpaug transformers sacremoses nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 25.3 MB/s eta 0:00:00


In [6]:
#단어 삽입
import nlpaug.augmenter.word as naw

texts = [
    "Those who can imagine anything, can create the impossible",
    "We can only see a short distance ahead, but we can see plenty there that needs to be done",
    "If a machine is expected to be infallible, it cannot also be intelligent",
]

#ContextualWordEmbsAug 클래스는 허깅페이스에서 BERT 모델을 불러온다.
#action은 모델이 수행할 기능을 선택, 해당 클래스는 subtitute 기능도 제공한다.
#입력 데이터는 리스트 구조로 반환된다.
aug = naw.ContextualWordEmbsAug(model_path = "distilbert-base-uncased", action="insert")
augmented_texts = aug.augment(texts)

for text, augmented in zip(texts, augmented_texts):
  print(f"src : {text}")
  print(f"dst : {augmented}")
  print("----------------")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

src : Those who can imagine anything, can create the impossible
dst : those living who can imagine anything, can spontaneously create the impossible impossible
----------------
src : We can only see a short distance ahead, but we can see plenty there that needs to be done
dst : we can only now see it a few short distance up ahead, perhaps but we already can see plenty there that needs to be done
----------------
src : If a machine is expected to be infallible, it cannot also be intelligent
dst : if otherwise a machine detector is expected to be highly infallible, it cannot therefore also not be intelligent
----------------


In [8]:
#문자 삭제
#삽입은 word로, 삭제는 char
import nlpaug.augmenter.char as nac

texts = [
    "Those who can imagine anything, can create the impossible",
    "We can only see a short distance ahead, but we can see plenty there that needs to be done",
    "If a machine is expected to be infallible, it cannot also be intelligent",
]

aug = nac.RandomCharAug(action = "delete")
augmented_texts = aug.augment(texts)

for text, augmented in zip(texts, augmented_texts):
  print(f"src : {text}")
  print(f"dst : {augmented}")
  print("--------------")

src : Those who can imagine anything, can create the impossible
dst : hos who can imge anything, can create the imoible
--------------
src : We can only see a short distance ahead, but we can see plenty there that needs to be done
dst : We can ly see a short tance ead, but we can see lnty the that needs to be dn
--------------
src : If a machine is expected to be infallible, it cannot also be intelligent
dst : If a main is expected to be infalle, it annt al be intlign
--------------


In [10]:
#단어 교체
import nlpaug.augmenter.word as naw

texts = [
    "Those who can imagine anything, can create the impossible",
    "We can only see a short distance ahead, but we can see plenty there that needs to be done",
    "If a machine is expected to be infallible, it cannot also be intelligent",
]

#RandomWordAug 클래스는 삽입, 대체, 교체, 삭제, 자르기 기능을 지원한다. (insert/subtitute/swap/delete/crop)
aug = naw.RandomWordAug(action = "swap")
augmented_texts = aug.augment(texts)

for text, augmented in zip(texts, augmented_texts):
  print(f"src : {text}")
  print(f"dst : {augmented}")
  print("--------------")

src : Those who can imagine anything, can create the impossible
dst : Those who imagine can anything, create can impossible the
--------------
src : We can only see a short distance ahead, but we can see plenty there that needs to be done
dst : Can we see a only short distance ahead, but we see can there plenty that to needs be done
--------------
src : If a machine is expected to be infallible, it cannot also be intelligent
dst : A if machine is expected to, be infallible cannot it also intelligent be
--------------


In [22]:
import nlpaug
print(nlpaug.__version__)
!pip install --upgrade nlpaug

1.1.11


In [3]:
#단어 대체(1)
#SynonymAug 클래스를 사용해 단어를 대체한다.
import nlpaug.augmenter.word as naw

texts = [
    "Those who can imagine anything, can create the impossible",
    "We can only see a short distance ahead, but we can see plenty there that needs to be done",
    "If a machine is expected to be infallible, it cannot also be intelligent",
]

#클래스는 wordnet 데이터베이스나 PPDB를 활용해 단어를 대체한다.
#ppdb의 경우 model_path 값을 지정해야 한다.
aug = naw.SynonymAug(aug_src="wordnet")
augmented_texts = aug.augment(texts)

for text, augmented in zip(texts, augmented_texts):
  print(f"src : {text}")
  print(f"dst : {augmented}")
  print("--------------")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


LookupError: 
**********************************************************************
  Resource [93maveraged_perceptron_tagger_eng[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('averaged_perceptron_tagger_eng')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtaggers/averaged_perceptron_tagger_eng/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [17]:
#단어 대체(2)
#ReservedAug 클래스를 사용해 단어를 대체한다.
import nlpaug.augmenter.word as naw

texts = [
    "Those who can imagine anything, can create the impossible",
    "We can only see a short distance ahead, but we can see plenty there that needs to be done",
    "If a machine is expected to be infallible, it cannot also be intelligent",
]

reserved_tokens = [
    ["can", "can't", "cannot", "could"],
]

reserved_aug = naw.ReservedAug(reserved_tokens = reserved_tokens)
augmented_texts = reserved_aug.augment(texts)

for text, augmented in zip(texts, augmented_texts):
  print(f"src : {text}")
  print(f"dst : {augmented}")
  print("--------------")

src : Those who can imagine anything, can create the impossible
dst : Those who could imagine anything, can't create the impossible
--------------
src : We can only see a short distance ahead, but we can see plenty there that needs to be done
dst : We could only see a short distance ahead, but we can't see plenty there that needs to be done
--------------
src : If a machine is expected to be infallible, it cannot also be intelligent
dst : If a machine is expected to be infallible, it can't also be intelligent
--------------


In [5]:
#역번역
import nlpaug.augmenter.word as naw

texts = [
    "Those who can imagine anything, can create the impossible",
    "We can only see a short distance ahead, but we can see plenty there that needs to be done",
    "If a machine is expected to be infallible, it cannot also be intelligent",
]

back_translation = naw.BackTranslationAug(
    from_model_name = "facebook/wmt19-en-de",
    to_model_name = "facebook/wmt19-de-en"
)

augmented_texts = back_translation.augment(texts)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/825 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.08G [00:00<?, ?B/s]

Some weights of FSMTForConditionalGeneration were not initialized from the model checkpoint at facebook/wmt19-en-de and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

...

vocab-src.json:   0%|          | 0.00/849k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/315k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

vocab-src.json:   0%|          | 0.00/849k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/315k [00:00<?, ?B/s]

In [6]:
for text, augmented in zip(texts, augmented_texts):
  print(f"src : {text}")
  print(f"dst : {augmented}")
  print("--------------")


src : Those who can imagine anything, can create the impossible
dst : If you can imagine anything, you can achieve the impossible
--------------
src : We can only see a short distance ahead, but we can see plenty there that needs to be done
dst : We can only take a brief look ahead, but we can see that there is still a lot to be done to be done to be done. We have to be done, and that is a lot of us, and that is a lot of us to be done, and that is a lot of us, and that is a lot of us, and that is to be done, and that is to be done, and that is to be done, and that is a lot of it is to be done, and that is to be done, and that is to be done, and that is to be done, and that is a lot of it, and that is to be done, and that is to be done, and that is to be done, is to be done, and that is to be done, is to be done, and that is to be done, is to be done, is to be done, is to be done, is to be done, is to be done, and that is to be done, and that is to be done, is to be done, is to be done,